In [1]:
import torch
import numpy
print(torch.__version__)

1.13.1


In [2]:
print(torch.version.cuda)

11.6


In [3]:
import navec

In [4]:
cuda = torch.device('cuda')

In [7]:
import csv
import sys

csv.field_size_limit(int(sys.maxsize / 100000000000))
texts = []
signs = []
dates = []
sites = []
with open('./signed.csv', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        texts.append(row[3])
        signs.append(row[8])
        dates.append(row[2])
        sites.append(row[4])
signs = signs[1:]
print(texts[1])
print(signs[1])
print(sites[1])
print(dates[1])
word_counts = []
for text in texts:
    words = text.split()
    word_counts.append(len(words))
print(sum(word_counts) / len(word_counts))

«ментовские войны» обошли приквел «игры престолов» пиратским ссылкам российские сериалы «тайны следствия» «ментовские войны» стали лидерами числу пиратских ссылок сети  опередив приквел «игры престолов»  аналитики объясняют это выходом новых сезонов «чисткой» другого нелегального контента кадр сериала «тайны следствия»  фото  студия «панорама»  начале      года проекты холдинга вгтрк «тайны следствия» «ментовские войны» возглавили рейтинг фильмов сериалов количеству ссылок  размещенных пиратских ресурсах  такие данные рбк представили компании contentscan  занимающейся защитой аудиовизуального контента нелегального распространения  первом квартале этого года аналитики contentscan выявили сети    тыс  ссылок пиратские ресурсы  где были доступны просмотра скачивания «тайны следствия»     тыс  ссылок «ментовскими войнами»  российские сериалы числу нелегальных ссылок обошли приквел «игры престолов» — проект «дом дракона»  который был доступен    тыс  пиратских сайтов  финальный эпизод этого

In [9]:
import numpy as np
#from gensim.models import Word2Vec
import time

#sentences = [text.split(' ') for text in texts]
#start = time.time()
#model = Word2Vec(sentences=sentences, vector_size=600, window=5, min_count=2, workers=20)
#model.save("word2vec.model")
#end = time.time()
#print(end - start)
from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [12]:
print(navec['газпром'])
print(len(navec['газпром']))

[ 1.82315037e-02 -1.81053311e-01  2.81231523e-01  2.17065945e-01
 -1.97169080e-01 -2.07226068e-01 -3.60865533e-01 -5.51898599e-01
 -8.80788192e-02  7.77008608e-02 -9.62384231e-03  1.15975894e-01
 -1.36517242e-01  1.29437640e-01 -5.46963036e-01  5.28605819e-01
 -7.17059791e-01  5.64617850e-02 -1.11396119e-01  1.44787505e-01
 -4.43636417e-01  1.03246018e-01 -2.97862738e-01 -3.75619978e-01
  1.53007030e-01 -1.82983324e-01  3.41885477e-01  1.96679264e-01
 -2.16666460e-01 -4.00866151e-01 -2.37134188e-01 -4.67940211e-01
 -1.65328875e-01 -2.06626505e-01 -3.20377797e-01 -2.40370944e-01
  1.18204214e-01  1.99869387e-02  4.00750518e-01 -2.18730625e-02
  2.08365247e-01  2.58554369e-01 -6.57887012e-02  5.91808081e-01
 -9.50129479e-02  5.16225137e-02  3.16363096e-01 -6.89696193e-01
  1.84347436e-01  1.59001067e-01  2.03682467e-01 -4.21593994e-01
 -2.19290227e-01 -6.32000208e-01 -4.73769605e-01 -1.71639159e-01
 -2.94465393e-01 -3.12448978e-01 -8.69688615e-02 -9.74455893e-01
 -7.83989489e-01  7.51606

In [14]:
vectors = [[navec[w] for w in text.split(' ') if w in navec] for text in texts[1:]]

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

X = vectors
classes = [-1, 0, 1]
for i in range(len(signs)):
    signs[i] = int(signs[i])
y = torch.zeros(len(vectors), 3, device='cuda:0')
for i in range(len(vectors)):
    y[i][classes.index(signs[i])] = 1

In [17]:
unique_date_and_sign = {(dates[i], signs[i]): i for i in range(len(X))}
indices = set(unique_date_and_sign.values())
Xr = [X[i] for i in range(len(X)) if i in indices]
yr = [y[i] for i in range(len(y)) if i in indices]
sitesr = [sites[i] for i in range(len(sites)) if i in indices]
print(len(Xr))
print(len(yr))

import random

def shuffle(X, y, sites):
    indices = [i for i in range(len(X))]
    random.shuffle(indices)
    X = [X[i] for i in indices]
    y = [y[i] for i in indices]
    sites = [sites[i] for i in indices]
    return (X, y, sites)
X, y, newSites = shuffle(X, y, sites)
Xr, yr, newSitesr = shuffle(Xr, yr, sitesr)

6175
6175


In [18]:
count = int(len(Xr) * 0.7)
val_count = int(len(Xr) * 0.8)
Xtrain, Xval, Xtest = Xr[:count], Xr[count:val_count], Xr[val_count:]
Ytrain, Yval, Ytest = yr[:count], yr[count:val_count], yr[val_count:]

In [19]:
Strain, Sval, Stest = newSitesr[:count], newSitesr[count:val_count], newSitesr[val_count:]

In [200]:
count = int(len(X) * 0.7)
val_count = int(len(X) * 0.8)
Xtrain, Xval, Xtest = X[:count], X[count:val_count], X[val_count:]
Ytrain, Yval, Ytest = y[:count], y[count:val_count], y[val_count:]

In [20]:
#TODO: change lstm, linear, relu to model
def accuracy(Xtrain, Ytrain, lstm, linear, relu):
    count = 0
    for i in range(len(Xtrain)):
        hidden = (
            torch.rand(2, 100, device='cuda:0'),
            torch.rand(2, 100, device='cuda:0')
        )
        x = Xtrain[i]
        x = np.array(x)
        out, _ = lstm(torch.cuda.FloatTensor(x, device='cuda:0'), hidden)
        out = linear(out[-1])
        out = relu(out)
        yPred = out
        yPred = yPred.reshape(-1)
        _max = 0
        j_max = 0
        for j in range(len(yPred)):
            if _max < yPred[j]:
                _max = yPred[j]
                j_max = j
        if Ytrain[i][j_max] == 1:
            count += 1
    return count / len(Xtrain)

In [ ]:
lstm = nn.LSTM(input_size=300, hidden_size=100, num_layers=2)
lstm.to("cuda:0")
linear = nn.Linear(100, 3)
linear.to("cuda:0")
#lstm = torch.load(f"lstmAdamWDefaultAcc85.model")
#linear = torch.load(f"linearAdamWDefaultAcc85.model")
relu = nn.Softmax(dim=0)
relu.to("cuda:0")
optimizer = optim.AdamW(lstm.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

lstm.train()
for epoch in range(1000):
    total_loss = 0
    global_start = time.time()
    for i in range(len(Xtrain)):
        optimizer.zero_grad()
        hidden = (
            torch.rand(2, 100, device='cuda:0'),
            torch.rand(2, 100, device='cuda:0')
        )
        x = Xtrain[i]
        x = np.array(x)
        out, _ = lstm(torch.cuda.FloatTensor(x, device='cuda:0'), hidden)
        out = linear(out[-1])
        out = relu(out)
        yPred = out
        loss = loss_fn(yPred, Ytrain[i]) / len(Xtrain)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    total_loss /= len(Xtrain)
    _accuracy = accuracy(Xtrain, Ytrain, lstm, linear, relu)
    val_accuracy = accuracy(Xval, Yval, lstm, linear, relu)
    global_end = time.time()
    if epoch % 10 == 0 and epoch != 0:
        torch.save(lstm, f"lstm{epoch}.model")
        torch.save(linear, f"linear{epoch}.model")
    print(f"Epoch {epoch}: Time: {global_end - global_start}s: Loss: {total_loss} Accuracy: {_accuracy}, Validation accuracy: {val_accuracy}")

Epoch 0: Time: 29.583858251571655s: Loss: 0.00024286888488667487 Accuracy: 0.24386857936140677, Validation accuracy: 0.24271844660194175
Epoch 1: Time: 25.776061058044434s: Loss: 0.0002302968605582127 Accuracy: 0.5821378991207774, Validation accuracy: 0.5647249190938511
Epoch 2: Time: 25.17223310470581s: Loss: 0.00021904151965716935 Accuracy: 0.6001850994909764, Validation accuracy: 0.5906148867313916
Epoch 3: Time: 26.132951974868774s: Loss: 0.00021656351691111492 Accuracy: 0.561082832022212, Validation accuracy: 0.5210355987055016
Epoch 4: Time: 24.971521139144897s: Loss: 0.00021490758040459626 Accuracy: 0.6066635816751504, Validation accuracy: 0.5631067961165048
Epoch 5: Time: 25.550198554992676s: Loss: 0.00021111723654013522 Accuracy: 0.6184636742248959, Validation accuracy: 0.5970873786407767
Epoch 6: Time: 26.22035264968872s: Loss: 0.00021100142445950305 Accuracy: 0.6298010180472003, Validation accuracy: 0.6213592233009708
Epoch 7: Time: 25.601808547973633s: Loss: 0.0002119105063

In [208]:
accuracy(Xtest, Ytest, lstm, linear, relu)

0.8342916522341531

In [203]:
torch.save(lstm, f"lstmAdamWLr0_01VecSize600Acc83_4.model")
torch.save(linear, f"linearAdamWLr0_01VecSize600Acc83_4.model")